In [1]:
import numpy as np

In [2]:
from dataframe import Data
from io_utils import *
import matplotlib.pyplot as plt

In [3]:
from pandas import DataFrame
from Predict_lc import PredictLightCurve
import fink_science
import os
from fink_science.utilities import load_scikit_model

In [4]:
from prediction_evaluation import PredictionEvaluation

In [5]:
data_ob = load_ztf_data(phot_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/ZTF_MSIP_MODEL51/ZTF_MSIP_NONIaMODEL0-0001_PHOT.FITS', drop_separators=True)

In [6]:
color_band_dict = {'g':'C2', 'r':'C3'}

data_ob.create_features_df(prediction_type_nos= [151,150],
                            band_choice='u', 
                            num_pc_components=3,
                            min_flux_threshold =200,
                           )

/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/biswajit/anaconda3/envs/kilonova/lib/python3.7/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))


In [25]:
print(data_ob.features_df[data_ob.features_df['id']==54301457])

           id  type        0pc1        0pc2        0pc3  0n         1pc1  \
125  54301457   151  397.511109 -262.858454  417.124596  17  1842.553056   

           1pc2         1pc3  1n  y_true  
125 -661.358113  1332.510016  16       1  


In [14]:
curdir = os.path.dirname(os.path.abspath(fink_science.__file__))
model = curdir + '/data/models/kilonova_model.pkl'
#model = '/home/biswajit/Kilonova/models/kn_model_new2.pkl' 
clf = load_scikit_model(model)

In [16]:
cols = []
bands = ['g','r']
num_pc_components =3
use_number_of_points = True
for i,band in enumerate(bands):
    for j in range(1,num_pc_components+1):
        if use_number_of_points:
            if j ==1:
                col_name = str(i)+'n'
                cols.append(col_name)
        col_name = str(i)+'pc'+str(j)
        cols.append(col_name)
        
print(cols)
        

['0n', '0pc1', '0pc2', '0pc3', '1n', '1pc1', '1pc2', '1pc3']


In [23]:
data_ob.features_df = data_ob.features_df[data_ob.features_df['id']==54301457]

In [26]:
data_ob.features_df

,id,type,0pc1,0pc2,0pc3,0n,1pc1,1pc2,1pc3,1n,y_true
125,54301457,151,397.511109,-262.858454,417.124596,17,1842.553056,-661.358113,1332.510016,16,1


In [24]:
print(data_ob.features_df[cols].values[0])

[  17.          397.5111091  -262.8584538   417.12459627   16.
 1842.55305592 -661.35811273 1332.51001608]


In [18]:
x_data = data_ob.features_df[cols].values
y_data = data_ob.features_df['y_true'].values

y_pred_data=clf.predict(x_data)
y_score= clf.predict_proba(x_data)

In [19]:
y_score

array([[0.64200752, 0.35799248],
       [0.66970978, 0.33029022],
       [0.74961184, 0.25038816],
       [0.35942322, 0.64057678],
       [0.48678888, 0.51321112],
       [0.72279379, 0.27720621],
       [0.88421034, 0.11578966],
       [0.49980722, 0.50019278],
       [0.27862824, 0.72137176],
       [0.45234541, 0.54765459],
       [0.50447818, 0.49552182],
       [0.28226825, 0.71773175],
       [0.69952463, 0.30047537],
       [0.5781252 , 0.4218748 ],
       [0.62224781, 0.37775219],
       [0.36408289, 0.63591711],
       [0.98885279, 0.01114721],
       [0.8459784 , 0.1540216 ],
       [0.74339095, 0.25660905],
       [0.79613912, 0.20386088],
       [0.73522743, 0.26477257],
       [0.37449461, 0.62550539],
       [0.68347271, 0.31652729],
       [0.28218723, 0.71781277],
       [0.33704212, 0.66295788],
       [0.33651239, 0.66348761],
       [0.33895284, 0.66104716],
       [0.59747076, 0.40252924],
       [0.32985173, 0.67014827],
       [0.79217124, 0.20782876],
       [0.

In [21]:
print(np.sum(y_pred_data))

80


In [29]:
import sklearn 
sklearn.metrics.confusion_matrix(y_pred_data, y_data)

array([[  0, 139],
       [  0,  80]])